# Build your own OpenAI Agent

With the [new OpenAI API](https://openai.com/blog/function-calling-and-other-api-updates) that supports function calling, it's never been easier to build your own agent!

In this notebook tutorial, we showcase how to write your own OpenAI agent in **under 50 lines of code**! It is minimal, yet feature complete (with ability to carry on a conversation and use tools).

## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the OpenAI API (using our own `llama_index` LLM class)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

In [1]:
import json
import os
import logging
from typing import Sequence, List

import openai
from dotenv import load_dotenv
from llama_index.llms import OpenAI, ChatMessage
from llama_index.tools import BaseTool, FunctionTool

import nest_asyncio

level = logging.WARNING

logger = logging.getLogger(__name__)
logging.basicConfig(level=level)
logger.setLevel(level)
logging.getLogger("llama_index").setLevel(level)
logging.getLogger("asyncio").setLevel(level)

load_dotenv("../../../.env")
assert (key := os.getenv("OPENAI_API_KEY"))
openai.api_key = key
nest_asyncio.apply()

Let's define some very simple calculator tools for our agent.

In [2]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [3]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

## Our (Slightly Better) `OpenAIAgent` Implementation 

We provide a (slightly better) `OpenAIAgent` implementation in LlamaIndex, which you can directly use as follows.  

In comparison to the simplified version above:
* it implements the `BaseChatEngine` and `BaseQueryEngine` interface, so you can more seamlessly use it in the LlamaIndex framework. 
* it supports multiple function calls per conversation turn
* it supports streaming
* it supports async endpoints
* it supports callback and tracing

In [4]:
from llama_index.agent import OpenAIAgent
from llama_index.agent.experimental_openai_agent import ExperimentalOpenAIAgent
from llama_index.llms import OpenAI

In [5]:
model = "gpt-3.5-turbo-0613"
#model = "gpt-4"
llm = OpenAI(model=model)
agent = OpenAIAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)
exp_agent = ExperimentalOpenAIAgent.from_tools(
    [multiply_tool, add_tool], llm=llm, verbose=True
)

### Chat

In [6]:
response = exp_agent.chat("What is (121 * 3) + 42?")
print(str(response))

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405
(121 * 3) + 42 is equal to 405.


In [7]:
# inspect sources
print(response.sources)

[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]


### Async Chat

In [8]:
response = await exp_agent.achat("What is 121 * 3?")
print(str(response))

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
121 * 3 is equal to 363.


### Streaming Chat
Here, every LLM response is returned as a generator. You can stream every incremental step, or only the last response.

In [ ]:
response = exp_agent.stream_chat(
    "What is 121 * 2? Once you have the answer, use that number to write a poem about a group of mice."
)

response_gen = response.response_gen

for ix, token in enumerate(response_gen):
    print(token, end="")

### Async Streaming Chat

In [9]:
response = await exp_agent.astream_chat(
    "What is 121 * 52 + 438? Once you have the answer, use that number to write a poem about a group of mice."
)

async for token in response.async_response_gen():
    print(token, end="")

assistant:  <class 'llama_index.llms.base.ChatResponse'>
assistant:  <class 'llama_index.llms.base.ChatMessage'>
MessageRole.ASSISTANT
assistant:  <class 'llama_index.llms.base.ChatResponse'>
assistant:  <class 'llama_index.llms.base.ChatMessage'>
MessageRole.ASSISTANT
assistant:  <class 'llama_index.llms.base.ChatResponse'>
assistant:  <class 'llama_index.llms.base.ChatMessage'>
MessageRole.ASSISTANT
assistant:  <class 'llama_index.llms.base.ChatResponse'>
assistant:  <class 'llama_index.llms.base.ChatMessage'>
MessageRole.ASSISTANT
assistant:  <class 'llama_index.llms.base.ChatResponse'>
assistant:  <class 'llama_index.llms.base.ChatMessage'>
MessageRole.ASSISTANT
assistant:  <class 'llama_index.llms.base.ChatResponse'>
assistant:  <class 'llama_index.llms.base.ChatMessage'>
MessageRole.ASSISTANT
assistant:  <class 'llama_index.llms.base.ChatResponse'>
assistant:  <class 'llama_index.llms.base.ChatMessage'>
MessageRole.ASSISTANT
assistant:  <class 'llama_index.llms.base.ChatResponse'

,assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a <class 'llama_index.llms.base.ChatResponse'>
assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a <class 'llama_index.llms.base.ChatMessage'>
MessageRole.ASSISTANT
assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group <class 'llama_index.llms.base.ChatResponse'>
assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group <class 'llama_index.llms.base.ChatMessage'>
MessageRole.ASSISTANT
assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of <class 'llama_index.llms.base.ChatResponse'>
assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of <class 'llama_index.llms.base.ChatMessage'>
MessageRole.ASSISTANT
 a group ofassistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of mice <class 'llama_index.llms.base.ChatResponse'>
assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little 

 perfectassistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of mice reside,
With tiny paws and whiskers, they scurry and hide.
Their numbers are many, a bustling community,
Living in harmony, in perfect unity <class 'llama_index.llms.base.ChatResponse'>
assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of mice reside,
With tiny paws and whiskers, they scurry and hide.
Their numbers are many, a bustling community,
Living in harmony, in perfect unity <class 'llama_index.llms.base.ChatMessage'>
MessageRole.ASSISTANT
assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of mice reside,
With tiny paws and whiskers, they scurry and hide.
Their numbers are many, a bustling community,
Living in harmony, in perfect unity.

 <class 'llama_index.llms.base.ChatResponse'>
assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of mice reside,
With tiny paws and whiskers, they scurry and hide.
Their n

assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of mice reside,
With tiny paws and whiskers, they scurry and hide.
Their numbers are many, a bustling community,
Living in harmony, in perfect unity.

They gather in the moonlight, under starry skies,
Sharing <class 'llama_index.llms.base.ChatResponse'>
assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of mice reside,
With tiny paws and whiskers, they scurry and hide.
Their numbers are many, a bustling community,
Living in harmony, in perfect unity.

They gather in the moonlight, under starry skies,
Sharing <class 'llama_index.llms.base.ChatMessage'>
MessageRole.ASSISTANT
assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of mice reside,
With tiny paws and whiskers, they scurry and hide.
Their numbers are many, a bustling community,
Living in harmony, in perfect unity.

They gather in the moonlight, under starry skies,
Sharing tales <class 'llama_index.ll

,assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of mice reside,
With tiny paws and whiskers, they scurry and hide.
Their numbers are many, a bustling community,
Living in harmony, in perfect unity.

They gather in the moonlight, under starry skies,
Sharing tales of adventure, with curious eyes.
With nimble feet, they <class 'llama_index.llms.base.ChatResponse'>
assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of mice reside,
With tiny paws and whiskers, they scurry and hide.
Their numbers are many, a bustling community,
Living in harmony, in perfect unity.

They gather in the moonlight, under starry skies,
Sharing tales of adventure, with curious eyes.
With nimble feet, they <class 'llama_index.llms.base.ChatMessage'>
MessageRole.ASSISTANT
 theyassistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of mice reside,
With tiny paws and whiskers, they scurry and hide.
Their numbers are many, a bustling commu

...

 creaturesassistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of mice reside,
With tiny paws and whiskers, they scurry and hide.
Their numbers are many, a bustling community,
Living in harmony, in perfect unity.

They gather in the moonlight, under starry skies,
Sharing tales of adventure, with curious eyes.
With nimble feet, they explore the world around,
Seeking crumbs and treasures, never to be found.

In the fields they frolic, through the meadows they roam,
Their tiny hearts beating, as they find their way home.
They build their nests with care, in secret nooks and crannies,
Creating a haven, where they can rest and be merry.

With each passing day, their numbers grow,
A testament to their resilience, a sight to behold.
They face challenges together, side by side,
Supporting one another, with love and pride.

So let us celebrate these creatures so <class 'llama_index.llms.base.ChatResponse'>
assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little co

 teachassistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of mice reside,
With tiny paws and whiskers, they scurry and hide.
Their numbers are many, a bustling community,
Living in harmony, in perfect unity.

They gather in the moonlight, under starry skies,
Sharing tales of adventure, with curious eyes.
With nimble feet, they explore the world around,
Seeking crumbs and treasures, never to be found.

In the fields they frolic, through the meadows they roam,
Their tiny hearts beating, as they find their way home.
They build their nests with care, in secret nooks and crannies,
Creating a haven, where they can rest and be merry.

With each passing day, their numbers grow,
A testament to their resilience, a sight to behold.
They face challenges together, side by side,
Supporting one another, with love and pride.

So let us celebrate these creatures so small,
The mice who teach us <class 'llama_index.llms.base.ChatResponse'>
assistant: 121 * 52 + 438 is equal to 6

 findassistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of mice reside,
With tiny paws and whiskers, they scurry and hide.
Their numbers are many, a bustling community,
Living in harmony, in perfect unity.

They gather in the moonlight, under starry skies,
Sharing tales of adventure, with curious eyes.
With nimble feet, they explore the world around,
Seeking crumbs and treasures, never to be found.

In the fields they frolic, through the meadows they roam,
Their tiny hearts beating, as they find their way home.
They build their nests with care, in secret nooks and crannies,
Creating a haven, where they can rest and be merry.

With each passing day, their numbers grow,
A testament to their resilience, a sight to behold.
They face challenges together, side by side,
Supporting one another, with love and pride.

So let us celebrate these creatures so small,
The mice who teach us, to stand tall.
In their unity and strength, we find inspiration <class 'llama_index.

 ourassistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of mice reside,
With tiny paws and whiskers, they scurry and hide.
Their numbers are many, a bustling community,
Living in harmony, in perfect unity.

They gather in the moonlight, under starry skies,
Sharing tales of adventure, with curious eyes.
With nimble feet, they explore the world around,
Seeking crumbs and treasures, never to be found.

In the fields they frolic, through the meadows they roam,
Their tiny hearts beating, as they find their way home.
They build their nests with care, in secret nooks and crannies,
Creating a haven, where they can rest and be merry.

With each passing day, their numbers grow,
A testament to their resilience, a sight to behold.
They face challenges together, side by side,
Supporting one another, with love and pride.

So let us celebrate these creatures so small,
The mice who teach us, to stand tall.
In their unity and strength, we find inspiration,
To cherish our own <

.assistant: 121 * 52 + 438 is equal to 6730.

In a cozy little corner, a group of mice reside,
With tiny paws and whiskers, they scurry and hide.
Their numbers are many, a bustling community,
Living in harmony, in perfect unity.

They gather in the moonlight, under starry skies,
Sharing tales of adventure, with curious eyes.
With nimble feet, they explore the world around,
Seeking crumbs and treasures, never to be found.

In the fields they frolic, through the meadows they roam,
Their tiny hearts beating, as they find their way home.
They build their nests with care, in secret nooks and crannies,
Creating a haven, where they can rest and be merry.

With each passing day, their numbers grow,
A testament to their resilience, a sight to behold.
They face challenges together, side by side,
Supporting one another, with love and pride.

So let us celebrate these creatures so small,
The mice who teach us, to stand tall.
In their unity and strength, we find inspiration,
To cherish our own bond

In [ ]:
raise Exception

### Agent with Personality

You can specify a system prompt to give the agent additional instruction or personality.

In [ ]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI
from llama_index.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
    system_prompt=SHAKESPEARE_WRITING_ASSISTANT,
)

In [ ]:
response = agent.chat("Hi")
print(response)

In [ ]:
response = agent.chat("Tell me a story")
print(response)